In [24]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    FewShotChatMessagePromptTemplate,
)

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    return_messages=True,
)

examples = [
    {
        "question": "What do you know about Movie Titanic?",
        "answer": """
        Titanic -> 🛳️💑😢
        """,
    },
    {
        "question": "What do you know about Movie The Godfather?",
        "answer": """
        The Godfather -> 👨‍👨‍👦🔫🍝
        """,
    },
    {
        "question": "What do you know about Movie Top Gun?",
        "answer": """
        Top Gun -> 🛩️👨‍✈️🔥
        """,
    },
]

template = [
    ("human", "{question}"),
    ("ai", "{answer}"),
]

example_prompt = ChatPromptTemplate.from_messages(template)

fewshot_chat_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie expert. Provide accurate 3 emogi images about movies when asked.",
        ),
        fewshot_chat_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


# '_' 는 이전에 반환한 값을 의미한다.
def load_memory(_):
    # print(_)
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})

    memory.save_context(
        {"input": question},
        {"output": str(result.content)},
    )
    print(result)


invoke_chain("What do you know about the movie the lion king.")

content='The Lion King -> 🦁👑🎶'


In [25]:
invoke_chain("What do you know about the movie star wars.")

content='Star Wars -> ⭐️🌌🚀'


In [28]:
invoke_chain("What do you know about the movie the Matrix")

content='Matrix -> 💻🕶️💥'


In [27]:
invoke_chain("What was the title of the last movie I asked you about?")

content='The last movie you asked me about was "Matrix."'
